# Results Summary

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
results_fm = pd.read_pickle("outputs/results_fm.pkl")
results_fm

In [ ]:
results_ros = pd.read_pickle("outputs/results_ros.pkl")
results_ros

In [ ]:
# Note, 167*3*12=6012
print(f"Total Time Periods: {len(results_ros.t.unique())}")
print(f"Earliest Train Start Time: {results_ros.t.min()}")
print(f"Latest Train Start Time: {results_ros.t.max()}")
print(f"Total Loss Functions: {len(results_ros.Loss.unique())}")
print(f"Total Models: {len(results_ros.Model.unique())}")
print(f"Total RMSE Calculations: {results_ros.shape[0]}")

In [ ]:
pd.options.display.float_format = '{:.3e}'.format

In [ ]:
tab1 = results_ros.groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab1

In [ ]:
print(tab1.to_latex(float_format='%.3e'))

In [ ]:
min_ros = tab1.Mean.argmin()
print(tab1.Loss[min_ros])
(tab1.Mean[0] - tab1.Mean[min_ros]) / tab1.Mean[0]

In [ ]:
min_ros

In [ ]:
print(f"Percent Reduction in RMSE for ROS from Best Exp. Loss: {100*(results_ros[results_ros.Loss == 'MSE'].RMSE.mean()- results_ros[results_ros.Loss == 'exp_0.0367'].RMSE.mean())/results_ros[results_ros.Loss == 'MSE'].RMSE.mean()}")

In [ ]:
print(f"Percent Reduction in RMSE for ROS from ROS Loss: {100*(results_ros[results_ros.Loss == 'MSE'].RMSE.mean()- results_ros[results_ros.Loss == 'ROS'].RMSE.mean())/results_ros[results_ros.Loss == 'MSE'].RMSE.mean()}")

In [ ]:
pd.options.display.float_format = None

In [ ]:
tab2 = results_fm.groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab2.round(2)

In [ ]:
#print(tab2.to_latex())

### Results by Model

In [ ]:
tab1 = results_ros[results_ros.Model == "XGB"].groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab1

In [ ]:
tab2 = results_fm[results_ros.Model == "XGB"].groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab2.round(2)

In [ ]:
tab1 = results_ros[results_ros.Model == "RF"].groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab1

In [ ]:
tab2 = results_fm[results_ros.Model == "RF"].groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab2.round(2)

In [ ]:
tab1 = results_ros[results_ros.Model == "LM"].groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab1

In [ ]:
tab2 = results_fm[results_ros.Model == "LM"].groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    Min=('RMSE', 'min'),
    Max=('RMSE', 'max')
).reset_index()
tab2.round(2)

## Significance Tests

### 2-Sample T-Test

Assumes indenpdent samples.

In [ ]:
from scipy import stats
stats.ttest_ind(results_ros[results_ros.Loss == "MSE"].RMSE, results_ros[results_ros.Loss == "exp_0.0367"].RMSE)

In [ ]:
stats.ttest_ind(results_fm[results_fm.Loss == "MSE"].RMSE, results_fm[results_fm.Loss == "exp_0.0367"].RMSE)

## Paired T-Test

Calculates t-test for related samples.

In [ ]:
# Check that time periods match, should be 1
print(np.mean(results_ros[results_ros.Loss == "MSE"].t == results_fm[results_ros.Loss == "exp_0.0367"].t))
# Check that Models are the same, should be 1
print(np.mean(results_ros[results_ros.Loss == "MSE"].Model == results_fm[results_ros.Loss == "exp_0.0367"].Model))

In [ ]:
len(results_ros[results_ros.Loss == "MSE"].RMSE)

In [ ]:
stats.ttest_rel(results_ros[results_ros.Loss == "MSE"].RMSE, results_ros[results_ros.Loss == "exp_0.0367"].RMSE)

In [ ]:
stats.ttest_rel(results_ros[results_ros.Loss == "MSE"].RMSE, results_ros[results_ros.Loss == "ROS"].RMSE)

In [ ]:
stats.ttest_rel(results_fm[results_fm.Loss == "MSE"].RMSE, results_fm[results_fm.Loss == "ROS"].RMSE)

In [ ]:
stats.ttest_rel(results_fm[results_fm.Loss == "MSE"].RMSE, results_fm[results_fm.Loss == "exp_0.0367"].RMSE)

## Plots

In [ ]:
# grouped = results_ros.groupby('Loss').agg({'RMSE': ['mean', 'sem']}).reset_index()
# grouped

grouped = results_fm.groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    SE=('RMSE', 'sem')
).reset_index()
grouped2 = results_ros.groupby('Loss', sort=False).agg(
    Mean=('RMSE', 'mean'),
    SE=('RMSE', 'sem')
).reset_index()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
ax1 = axes[0]
ax1.errorbar(grouped.Loss, grouped.Mean, yerr=grouped.SE, fmt='o', label='SE')
# plt.errorbar(tab1.Loss, tab1.Mean, yerr=[tab1.Mean[i] - tab1.Max[i] for i in range(len(tab1.Loss))], fmt='o', label='Min')
ax1.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax1.set_xlabel('Loss Function')
ax1.set_ylabel('Mean RMSE on FMC')
ax1.set_title('Mean RMSE by Loss Function (FMC)')
ax1.tick_params(axis='x', rotation=90)
ax1.axvline(x=10.5, color='black', linestyle='--')
ax1.grid()

ax2 = axes[1]
ax2.errorbar(grouped2.Loss, grouped2.Mean, yerr=grouped2.SE, fmt='o', label='SE', color='r')
# plt.errorbar(tab1.Loss, tab1.Mean, yerr=[tab1.Mean[i] - tab1.Max[i] for i in range(len(tab1.Loss))], fmt='o', label='Min')
ax2.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax2.set_xlabel('Loss Function')
ax2.set_ylabel('Mean RMSE on ROS')
ax2.set_title('Mean RMSE by Loss Function (ROS)')
ax2.tick_params(axis='x', rotation=90)
ax2.axvline(x=10.5, color='black', linestyle='--')
ax2.grid()

plt.tight_layout()
plt.savefig('outputs/results1.png')
plt.show()

In [ ]:
models_vec = results_ros.Model.unique()

In [ ]:
N = len(models_vec) # number of rows of subplots
fig, axes = plt.subplots(N, 2, figsize=(12, 5*N))

for i in range(0, len(models_vec)):
    mod = models_vec[i]
    temp = results_ros[results_ros.Model == mod]
    temp2 = temp.groupby('Loss', sort=False).agg(
        Mean=('RMSE', 'mean'),
        SE=('RMSE', 'sem')
    ).reset_index()
    # Access the subplot at row i, column 0
    ax1 = axes[i, 0]
    # ax1.scatter(temp2.Loss, temp2.Mean)
    ax1.errorbar(temp2.Loss, temp2.Mean, yerr=temp2.SE, fmt='o', label='SE')
    ax1.tick_params(axis='x', rotation=90)
    ax1.set_title(f'Test RMSE on FMC - Model {mod}')
    ax1.axvline(x=10.5, color='black', linestyle='--')
    ax1.grid()

    temp = results_fm[results_fm.Model == mod]
    temp2 = temp.groupby('Loss', sort=False).agg(
        Mean=('RMSE', 'mean'),
        SE=('RMSE', 'sem')
    ).reset_index()
    # Access the subplot at row i, column 0
    ax2 = axes[i, 1]
    ax2.errorbar(temp2.Loss, temp2.Mean, yerr=temp2.SE, fmt='o', label='SE', color='r')
    ax2.tick_params(axis='x', rotation=90)
    ax2.set_title(f'Test RMSE on ROS - Model {mod}')
    ax2.axvline(x=10.5, color='black', linestyle='--')
    ax2.grid()

plt.tight_layout()
plt.savefig('outputs/results2.png')
plt.show()